As a best practice, OpenFF software attempts to associate explicit units with numerical values. At times this seems overly cautious, but mistakes with units have caused crashes of everything from molecular simulations on silicon to [expensive rockets](https://en.wikipedia.org/wiki/Mars_Climate_Orbiter#Cause_of_failure) on Mars.

Units are tagged using the [`openff-units`](https://github.com/openforcefield/openff-units#openff-units) package, which provides numerical types associated with commonly used units and methods for ergonomically and safely converting between units. It is based on [Pint](https://github.com/hgrecco/pint), which behaves similarly to OpenMM's unit package.

In [ ]:
from openff.units import Quantity, unit

bond_length = Quantity(0.145, unit.nanometer)
bond_length

As a shorthand, you can create `Quantity` objects by multiplying numerical objects (int, float, `numpy.ndarray`, etc.) by a `unit.Unit` object, i.e.:

In [ ]:
value1 = Quantity(0.8, unit.kilocalorie_per_mole)
value2 = 0.8 * unit.kilocalorie_per_mole

assert value1 == value2

value1, value2

The `Quantity` class is provided by this `unit` namespace (and re-exported at the top-level `openff.units` module) so you will sometimes see it written as `unit.Quantity`:

In [ ]:
assert Quantity(0.5, unit.kilojoule_per_mole) == unit.Quantity(
    0.5, unit.kilojoule_per_mole
)
assert Quantity is unit.Quantity

These objects support conversion to other units via `Quantity.to()`, which takes another (compatible) unit as an argument:

In [ ]:
bond_length.to(unit.angstrom)

These objects can be converted to unitless representation as-is via `Quantity.magnitude` (or its alias `Quantity.m`) or with an added conversion via `Quantity.m_as`.

In [ ]:
bond_length.magnitude, bond_length.m_as(unit.angstrom), bond_length.m_as(unit.nanometer)

Scalar quantities can be serialized to strings using the built-in `str()` function and deserialized using the `Quantity` constructor. This is handy for reading and writing things like force fields - this is how OpenFF Toolkit interacts with SMIRNOFF (`.offxml`) force field files on disk.

In [ ]:
k = 10 * unit.kilocalorie / unit.mol / unit.nanometer**2
str(k)

In [ ]:
Quantity(str(k))

<div class="alert alert-warning" style="max-width: 700px; margin-left: auto; margin-right: auto;">
    ⚠️ <b> OpenFF and OpenMM unit packages are not directly interoperable!</b><br />
Passing <code>openff.units.Quantity</code> objects to the OpenMM API, or vice versa, may fail or silently produce surprising results such as stripping units without warning. Mixing objects from different unit packages (i.e. atomic positions represented by a combination of both objects) can also lead to confusing error messages.
</div>

For [compatibility with OpenMM units](https://github.com/openforcefield/openff-units#openmm-interoperability), a submodule (`openff.units.openmm`) with conversion functions (to_openmm, from_openmm) is also provided.

In [ ]:
from openff.units.openmm import from_openmm, to_openmm

distance = 4.0 * unit.femtosecond
converted = to_openmm(distance)
converted, type(converted)

In [ ]:
roundtripped = from_openmm(converted)
roundtripped, type(roundtripped)

For cases in which a quantity might be _either_ an `openff.units.Quantity` or `openmm.unit.Quantity`, a helper function `ensure_quantity` is provided to safely converted either into a specified type. Physically equivalent values provided by either units package should be processed into the same results if provided. The variables `distance`, `converted`, and `roundtripped` that we just made can be coerced into equivalent objects if requested.

In [ ]:
from openff.units.openmm import ensure_quantity

?ensure_quantity

In [ ]:
type(ensure_quantity(distance, "openff")), type(ensure_quantity(distance, "openmm"))

In [ ]:
assert ensure_quantity(distance, "openmm") == ensure_quantity(converted, "openmm")
assert ensure_quantity(distance, "openff") == ensure_quantity(converted, "openff")

An effort is made to convert from OpenMM constructs, such as converting `List[openmm.Vec3]` to wrapped NumPy arrays:

In [ ]:
import openmm.app

positions = openmm.app.PDBFile("../pdb/MainChain_ALA.pdb").getPositions()
type(positions), positions

In [ ]:
converted = from_openmm(positions)
type(converted), type(converted.m), converted

The OpenFF Toolkit uses `openff.units` for handling unit-bearing quantities as of release 0.11.0 (August 2022).

In [ ]:
from openff.toolkit import Molecule, __version__
from packaging.version import Version

assert Version(__version__) >= Version("0.11")

molecule = Molecule.from_smiles("c1ccc(Br)cc1Cl")
molecule

In [ ]:
molecule.generate_conformers(n_conformers=1)
molecule.assign_partial_charges(partial_charge_method="am1bcc")
type(molecule.conformers[0]), type(molecule.partial_charges)

In [ ]:
molecule.conformers

<div class="alert alert-info" style="max-width: 700px; margin-left: auto; margin-right: auto;">
ℹ️ Tip: If using OpenFF tools alongside OpenMM, it is useful to import their different units solutions with different names. Avoid importing both as <code>unit</code> in order to keep them separate in your workspace.</div>

In [ ]:
import openmm.unit
from openff.units import unit

In [ ]:
type(unit.angstrom * 1.2), type(openmm.unit.angstrom * 1.2)